In [1]:
import torchxrayvision as xrv

C:\Users\Hoo Kai Sng\OneDrive - National University of Singapore\Documents\NUS\CS3244\Project\xrayenv\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
data = xrv.datasets.COVID19_Dataset(imgpath = 'data/images/', csvpath = 'data/csv/metadata.csv')

In [5]:
p = 0
for i in range(len(data)):
    _, lab = data[i]['img'][0], data[i]['lab'][3]
    p += lab
print(len(data), p)

535 342.0
